In [336]:
%matplotlib inline
import requests
import bs4
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import urllib.request
import time

In [326]:
imdb_url = 'https://www.imdb.com/name/nm0000168/?ref_=tt_cl_t5'

In [327]:
response = requests.get(imdb_url)

In [328]:
soup = bs4.BeautifulSoup(response.text)

In [329]:
movies = []
year = []
ids = []

In [330]:
for instance in soup.findAll(class_ = re.compile('filmo-row .+'), id = re.compile('actor-\w*')):
    if len(re.findall('[12][0-9]{3}', instance.find('span').text)) == 0 or float(re.findall('[12][0-9]{3}', instance.find('span').text)[0]) > 2019:
        continue
    if len(re.findall('(TV)|([sS]hort)|([vV]ideo)|([gG]ame)', instance.text)) > 0:
        continue
    if instance.find('b'):
        movies.append(instance.find('b').text)
    ids.append(re.findall('tt[0-9]*', instance.get('id'))[0])
    if instance.find('span'):
        year.append(re.findall('[12][0-9]{3}', instance.find('span').text)[0])

In [331]:
sammyj_df = pd.DataFrame()

In [332]:
sammyj_df['Movie'] = movies
sammyj_df['Year'] = year
sammyj_df['id'] = ids

In [333]:
sammyj_df.head()

,Movie,Year,id
0,Spider-Man: Far from Home,2019,tt6320628
1,Shaft,2019,tt4463894
2,Captain Marvel,2019,tt4154664
3,Glass,2019,tt6823368
4,Life Itself,2018,tt5989218


In [341]:
# Fetch the box office numbers
dates = []
total_domestic = []
total_international = []
total = []
delete_ids = []

imdb_id = ids[0]
for movie in ids:
    time.sleep(1)
    base = 'https://www.boxofficemojo.com'
    url = base + '/title/' + movie
    soup = bs4.BeautifulSoup(requests.get(url).text)
    if not soup.findAll(class_ = 'a-bordered a-horizontal-stripes a-size-base-plus'):
        delete_ids.append(movie)
        dates.append('delete')
        total_domestic.append('delete')
        total_international.append('delete')
        total.append('delete')
        continue
    cool = soup.findAll(class_ = 'a-bordered a-horizontal-stripes a-size-base-plus')[0].findAll('td')
    date = cool[1].text
    domestic = cool[3].text
    international = cool[4].text
    total_bo = cool[5].text
    
    dates.append(date)
    if not '$' in domestic:
        total_domestic.append(0)
    else:
        total_domestic.append(domestic)
        
    if not '$' in international:
        total_international.append(0)
    else:
        total_international.append(international)
    total.append(total_bo)




In [350]:
dates

['June 28-July 10, 2019',
 'June 14, 2019',
 'March 6-15, 2019',
 'January 16-18, 2019',
 'September 21-March 8, 2019',
 'June 14-September 27, 2018',
 'April 25-May 11, 2018',
 'delete',
 'August 16-October 12, 2017',
 'March 8-25, 2017',
 'January 18-February 24, 2017',
 'September 28-February 4, 2017',
 'June 29-July 30, 2016',
 'June 29-October 6, 2016',
 'December 25-February 27, 2016',
 'December 4-December 2, 2016',
 'April 15-November 24, 2016',
 'April 22-July 4, 2015',
 'January 30-September 11, 2015',
 'June 12-January 30, 2015',
 'March 26-April 19, 2014',
 'January 30-May 7, 2014',
 'March 6-June 11, 2014',
 'January 3-September 18, 2014',
 'July 10-October 25, 2013',
 'December 25-May 12, 2013',
 'August 23, 2012-May 22, 2014',
 'April 25-August 17, 2012',
 'May 4, 2012',
 'May 18-August 23, 2012',
 'July 22-January 20, 2012',
 'April 21-November 3, 2011',
 'delete',
 'August 6-December 3, 2010',
 'May 26-August 5, 2011',
 'April 28-June 12, 2010',
 'October 10-September 

In [343]:
sammyj_df['Domestic'] = total_domestic
sammyj_df['International'] = total_international
sammyj_df['Total'] = total
sammyj_df['Dates'] = dates

In [363]:
sammyj_ids_df = sammyj_df.set_index('id')
sammyj_ids_df = sammyj_ids_df.drop(delete_ids)

In [372]:
def find_dates(date_string):
    return re.findall('[A-Z][a-z]+ [1-9]{1,2}', date_string)[0]

In [373]:
def find_season(month):
    if 'January' in month:
        return 'Winter'
    elif 'February' in month:
        return 'Winter'
    elif 'March' in month:
        return 'Spring'
    elif 'April' in month:
        return 'Spring'
    elif 'May' in month:
        return 'Spring'
    elif 'June' in month:
        return 'Summer'
    elif 'July' in month:
        return 'Summer'
    elif 'August' in month:
        return 'Summer'
    elif 'September' in month:
        return 'Fall'
    elif 'October' in month:
        return 'Fall'
    elif 'November' in month:
        return 'Fall'
    else:
        return 'Winter'

In [383]:
def find_year(date_string):
    return re.findall('[12][091][0-9]{2}', date_string)[0]

In [375]:
sammyj_ids_df['Release Date'] = sammyj_ids_df['Dates'].apply(find_dates)

In [376]:
sammyj_ids_df['Season of Release'] = sammyj_ids_df['Release Date'].apply(find_season)

In [384]:
sammyj_ids_df['Year of Release'] = sammyj_ids_df['Dates'].apply(find_year)

In [386]:
sammyj_ids_df.head()

,Movie,Year,Domestic,International,Total,Dates,Release Date,Season of Release,Year of Release
id,,,,,,,,,
tt6320628,Spider-Man: Far from Home,2019,"$390,532,085","$741,396,434","$1,131,928,519","June 28-July 10, 2019",June 28,Summer,2019
tt4463894,Shaft,2019,"$21,360,215",0,"$21,360,215","June 14, 2019",June 14,Summer,2019
tt4154664,Captain Marvel,2019,"$426,829,839","$701,444,955","$1,128,274,794","March 6-15, 2019",March 6,Spring,2019
tt6823368,Glass,2019,"$111,035,005","$135,950,571","$246,985,576","January 16-18, 2019",January 16,Winter,2019
tt5989218,Life Itself,2018,"$4,102,648","$3,687,623","$7,790,271","September 21-March 8, 2019",September 21,Fall,2019


In [403]:
sammyj_ids_df[['Domestic', 'International', 'Total']] = sammyj_ids_df[['Domestic', 'International', 'Total']].replace('[\$,]', '', regex=True).astype(float)

In [405]:
sammyj_ids_df.to_csv('sammyj_data.csv')

In [412]:
(sammyj_ids_df.groupby('Season of Release')['Total'].mean()/1000).to_csv('Total Mean Gross.csv')

In [409]:
sammyj_ids_df.groupby('Season of Release')['Movie'].count()

Season of Release
Fall      23
Spring    34
Summer    36
Winter    22
Name: Movie, dtype: int64

In [414]:
sammyj_ids_df.groupby('Season of Release')['Total'].idxmax()

Season of Release
Fall      tt0317705
Spring    tt4154756
Summer    tt3606756
Winter    tt1853728
Name: Total, dtype: object

In [422]:
sammyj_ids_df.loc[['tt0317705', 'tt4154756' ,'tt3606756', 'tt1853728']]

,Movie,Year,Domestic,International,Total,Dates,Release Date,Season of Release,Year of Release
id,,,,,,,,,
tt0317705,The Incredibles,2004,261441092.0,3.700010e+08,6.314421e+08,"November 5-February 3, 2005",November 5,Fall,2005
tt4154756,Avengers: Infinity War,2018,678815482.0,1.369544e+09,2.048360e+09,"April 25-May 11, 2018",April 25,Spring,2018
tt3606756,Incredibles 2,2018,608581744.0,6.342236e+08,1.242805e+09,"June 14-September 27, 2018",June 14,Summer,2018
tt1853728,Django Unchained,2012,162805434.0,2.625628e+08,4.253682e+08,"December 25-May 12, 2013",December 25,Winter,2013


In [423]:
sammyj_ids_df['Total'].idxmax()

'tt4154756'

In [429]:
sammyj_ids_df.drop([['tt4154756','tt0848228']], axis = 0)['Total'].idxmax()

KeyError: "[('tt4154756', 'tt0848228')] not found in axis"

In [428]:
sammyj_ids_df.loc[['tt0848228','tt4154756']]

,Movie,Year,Domestic,International,Total,Dates,Release Date,Season of Release,Year of Release
id,,,,,,,,,
tt0848228,The Avengers,2012,623357910.0,8.954551e+08,1.518813e+09,"April 25-August 17, 2012",April 25,Spring,2012
tt4154756,Avengers: Infinity War,2018,678815482.0,1.369544e+09,2.048360e+09,"April 25-May 11, 2018",April 25,Spring,2018


In [433]:
sammyj_no_avengers = sammyj_ids_df.sort_values('Total', ascending = False).drop(['tt4154756', 'tt0848228', 'tt2395427'])

In [436]:
(sammyj_no_avengers.groupby('Season of Release')['Total'].mean()/1000).to_csv('no_top_three.csv')

In [441]:
total_by_year = sammyj_ids_df.groupby('Year')['Total'].mean()

In [454]:
eighties = re.findall('198[0-9]', total_by_year.apply('str'))
nineties = re.findall('199[0-9]', total_by_year.apply('str'))
two_thousands = re.findall('200[0-9]', total_by_year.apply('str'))
two_thousand_tens = re.findall('201[0-9]', total_by_year.apply('str'))

In [459]:
total_eighties = total_by_year.loc[eighties].mean()
total_nineties = total_by_year.loc[nineties].mean()
total_two_thousands = total_by_year.loc[two_thousands].mean()
total_two_thousand_tens = total_by_year.loc[two_thousand_tens].mean()

C:\Users\richa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [467]:
mean_gross_total_decade = pd.Series()

In [468]:
mean_gross_total_decade['1980\'s'] = total_eighties
mean_gross_total_decade['1990\'s'] = total_nineties
mean_gross_total_decade['2000\'s'] = total_two_thousands
mean_gross_total_decade['2010\'s'] = total_two_thousand_tens

In [470]:
(mean_gross_total_decade/1000).to_csv('total_gross_by_decade.csv')

In [473]:
(total_by_year/1000).to_csv('mean_total_by_year.csv')